In [1]:
from fastai.text import *

In [2]:
# find fastai data path
#path = Config().data_path(); path

In [3]:
#! wget https://s3.amazonaws.com/fast-ai-nlp/giga-fren.tgz -P {path}

In [4]:
#! tar xf {path}/giga-fren.tgz -C {path}

In [5]:
path = Config().data_path()/'giga-fren'
path.ls()

[PosixPath('/home/bf/.fastai/data/giga-fren/data_save.pkl'),
 PosixPath('/home/bf/.fastai/data/giga-fren/cc.fr.300.bin'),
 PosixPath('/home/bf/.fastai/data/giga-fren/cc.en.300.bin'),
 PosixPath('/home/bf/.fastai/data/giga-fren/giga-fren.release2.fixed.en'),
 PosixPath('/home/bf/.fastai/data/giga-fren/giga-fren.release2.fixed.fr'),
 PosixPath('/home/bf/.fastai/data/giga-fren/questions_easy.csv')]

In [ ]:
# with open(path/'giga-fren.release2.fixed.fr') as f: 
#     fr = f.read(100000).split('\n')

In [ ]:
# with open(path/'giga-fren.release2.fixed.en') as f: 
#     en = f.read(100000).split('\n')

In [ ]:
# re_eq = re.compile('^(Wh[^?.!]+\?)')
# re_fq = re.compile('^([^?.!]+\?)')
# en_fname = path/'giga-fren.release2.fixed.en'
# fr_fname = path/'giga-fren.release2.fixed.fr'

In [ ]:
# lines = ((re_eq.search(eq), re_fq.search(fq)) 
#         for eq, fq in zip(open(en_fname, encoding='utf-8'), open(fr_fname, encoding='utf-8')))
# qs = [(e.group(), f.group()) for e,f in lines if e and f]

In [ ]:
# qs = [(q1,q2) for q1,q2 in qs]
# df = pd.DataFrame({'fr': [q[1] for q in qs], 'en': [q[0] for q in qs]}, columns = ['en', 'fr'])
# df.to_csv(path/'questions_easy.csv', index=False)

In [ ]:
#path.ls()

In [ ]:
# Load data
df = pd.read_csv(path/'questions_easy.csv')
df.head()

In [ ]:
# lower case
df['en'] = df['en'].apply(lambda x: x.lower())
df['fr'] = df['fr'].apply(lambda x: x.lower())

In [6]:
def seq2seq_collate(samples, pad_idx=1, pad_first=True, backwards=False):
    "Function that collect samples and adds padding. Flips token order if needed"
    samples = to_data(samples)
    # max sentence length
    max_len_x,max_len_y = max([len(s[0]) for s in samples]),max([len(s[1]) for s in samples])
    # default by padding
    res_x = torch.zeros(len(samples), max_len_x).long() + pad_idx
    res_y = torch.zeros(len(samples), max_len_y).long() + pad_idx
    # backwards = not pad_first
    if backwards: pad_first = not pad_first
    for i,s in enumerate(samples):
        if pad_first: # pad left
            res_x[i,-len(s[0]):],res_y[i,-len(s[1]):] = LongTensor(s[0]),LongTensor(s[1])
        else: #pad right     
            res_x[i,:len(s[0]):],res_y[i,:len(s[1]):] = LongTensor(s[0]),LongTensor(s[1])
    if backwards: res_x,res_y = res_x.flip(1),res_y.flip(1)
    return res_x,res_y

In [7]:
#doc(Dataset)

In [8]:
#doc(DataLoader)

In [9]:
#doc(DataBunch)

In [7]:
class Seq2SeqDataBunch(TextDataBunch):
    "Create a `TextDataBunch` suitable for training an RNN classifier."
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None,
              path:PathOrStr=".", bs:int=32, val_bs:int=None,
               pad_idx=1, dl_tfms=None, pad_first=False,
               device:torch.device=None,
              no_check:bool=False, backwards:bool=False, 
               **dl_kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds,test_ds)
        val_bs = ifnone(val_bs, bs) # val_size
        collate_fn = partial(seq2seq_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs//2)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
            return cls(*dataloaders, path=path, device=device, collate_fn=collate_fn, no_check=no_check)

In [11]:
#SortishSampler??
# sort thing with a little bit of randomness

In [8]:
class Seq2SeqTextList(TextList):
    _bunch = Seq2SeqDataBunch
    _label_cls = TextList

In [ ]:
src = Seq2SeqTextList.from_df(df, path = path, cols='fr').split_by_rand_pct(seed=42).label_from_df(cols='en', label_cls=TextList)

In [ ]:
# 90 percentile of x
np.percentile([len(o) for o in src.train.x.items] + [len(o) for o in src.valid.x.items], 90)

In [23]:
# 90 percentile of y
np.percentile([len(o) for o in src.train.y.items] + [len(o) for o in src.valid.y.items], 90)

23.0

In [24]:
# remove items > 30
src = src.filter_by_func(lambda x,y: len(x)>30 or len(y)>30)

In [25]:
len(src.train)+len(src.valid)

48350

In [26]:
data = src.databunch()

In [27]:
#data.save()

In [28]:
data

Seq2SeqDataBunch;

Train: LabelList (38704 items)
x: Seq2SeqTextList
xxbos qu’est - ce que la lumière ?,xxbos où sommes - nous ?,xxbos d'où venons - nous ?,xxbos que ferions - nous sans elle ?,xxbos quel est le groupe autochtone principal sur l’île de vancouver ?
y: TextList
xxbos what is light ?,xxbos who are we ?,xxbos where did we come from ?,xxbos what would we do without it ?,xxbos what is the major aboriginal group on vancouver island ?
Path: /home/bf/.fastai/data/giga-fren;

Valid: LabelList (9646 items)
x: Seq2SeqTextList
xxbos quels pourraient être les effets sur l’instrument de xxunk et sur l’aide humanitaire qui ne sont pas co - xxunk ?,xxbos quand la source primaire a - t - elle été créée ?,xxbos pourquoi tant de soldats ont - ils fait xxunk de ne pas voir ce qui s'est passé le 4 et le 16 mars ?,xxbos quels sont les taux d'impôt sur le revenu au canada pour 2007 ?,xxbos pourquoi le programme devrait - il intéresser les employeurs et les fournisseurs de services ?
y: TextLis

In [15]:
path

PosixPath('/home/bf/.fastai/data/giga-fren')

In [9]:
data = load_data(path)

In [10]:
data.show_batch()

text,target
xxbos quel est le coût ( total pour le gouvernement du canada ) par xxunk protégé et comment cela se compare - t - il à d’autres initiatives similaires ?,"xxbos what is the cost ( total , to goc ) per xxunk protected and how does this compare with other , similar initiatives ?"
xxbos quel est le lien entre la situation socio - démographique des femmes et leurs pratiques multiples sur les plans des risques et de la promotion de la santé ?,xxbos what is the relation between women ’s socio - demographic status and their multiple xxunk and multiple health - promoting practices ?
xxbos quel a été le rôle xxunk et agroalimentaire canada ( aac ) dans le processus de révision de la norme nationale sur l'agriculture biologique qui date de 1999 ?,xxbos what was the role of agriculture and agri - food canada ( aafc ) in the initiative to revise the 1999 national standard for organic agriculture ?
"xxbos que peuvent faire les enseignants pour créer un climat différent , un climat où les étudiants se sentent en sécurité , soutenus et en mesure xxunk des défis ?","xxbos what can teachers do to create a different climate , one in which learners feel safe , supported and challenged ?"
"xxbos à quel moment peut - on dire que la mise en oeuvre d'un système d'information est réussie , et quelle est la meilleure façon de mesurer cette réussite ?",xxbos what is a successful implementation of an information system and what is the best way to measure it ?


# Fasttext

In [11]:
import fasttext as ft
from pathlib import Path

In [12]:
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P {path}
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz -P {path}

In [13]:
# ! cd {path} && gunzip cc.en.300.bin.gz

In [14]:
# ! cd {path} && gunzip cc.fr.300.bin.gz

In [15]:
# !gunzip {path} / cc.en.300.bin.gz
# !gunzip {path} / cc.fr.300.bin.gz

In [16]:
# path = Path('/home/bf/.fastai/data/giga-fren')

In [17]:
def create_emb(vecs, itos, em_sz=300, mult=1.):
    emb = nn.Embedding(len(itos), em_sz, padding_idx=1)
    wgts = emb.weight.data
    # word:embedding
    vec_dic = {w:vecs.get_word_vector(w) for w in vecs.get_words()}
    miss = []
    for i,w in enumerate(itos):
        # emb.weight.data[i] = tensor(embedding_from ft)
        try: wgts[i] = tensor(vec_dic[w])
        except: miss.append(w)
    return emb

In [ ]:
model_path = Config().model_path()
fr_vecs = ft.load_model(str((path/'cc.fr.300.bin')))
emb_enc = create_emb(fr_vecs, data.x.vocab.itos)
torch.save(emb_enc, model_path/'fr_emb.pth')
del fr_vecs

In [ ]:
en_vecs = ft.load_model(str((path/'cc.en.300.bin')))
emb_dec = create_emb(en_vecs, data.y.vocab.itos)
torch.save(emb_dec, model_path/'en_emb.pth')
del en_vecs

In [ ]:
emb_enc.weight.size(), emb_dec.weight.size()

In [16]:
model_path = Config().model_path()

# Model

In [ ]:
class Seq2SeqRNN(nn.Module):
    def __init__(self, emb_enc, emb_dec,
                nh, out_sl, nl=2, bos_idx=0, pad_idx=1):
        super().__init__()
        self.nl, self.nh, self.out_sl = nl,nh,out_sl
        self.bos_idx, self.pad_idx = bos_idx, pad_idx
        self.em_sz_enc = emb_enc.embedding_dim
        self.em_sz_dec = emb_dec.embedding_dim
        self.voc_sz_dec = emb_dec.num_embeddings
        self.emb_enc = emb_enc
        self.emb_enc_drop = nn.Dropout(0.15)
        # gru, enc_dim > nh
        # output = bs, nh
        # h = bs,nh
        self.gru_enc = nn.GRU(self.em_sz_enc, nh, num_layers=nl,
                             dropout=0.25, batch_first=True)
        # out_enc, nh > 300
        # hid = bs, 300
        self.out_enc = nn.Linear(nh, self.em_sz_dec, bias=False)
        self.emb_dec = emb_dec
        # gru_dec, emb_dim > emb_dim
        # outp = bs, out_sl, 300
        # hid = bs, 300
        self.gru_dec = nn.GRU(self.em_sz_dec, self.em_sz_dec, num_layers=nl,
                             dropout=0.1, batch_first=True)
        self.out_drop= nn.Dropout(0.35)
        self.out = nn.Linear(self.em_sz_dec, self.voca_sz_dec)
        self.out.weight.data = self.emb_dec.weight.data
    
    def encoder(self,bs,inp):
        h = self.initHidden(bs)
        emb = self.emb_enc_drop(self.emb_enc(inp))
        _,h= self.gru_enc(emb, h) #h=bs,nh
        h = self.out_enc(h) #h = bs,300
        return h
    
    def decoder(self, dec_inp, h):
        emb = self.emb_dec(dec_inp).unsqueeze(1)
        outp, h = self.gru_dec(emb, h)# use the encoder h > (bs,300)
        return h, outp
    
    def forward(self, inp):
        bs, sl = inp.size()
        h = self.encoder(bs, inp)
        dec_inp = inp.new_zeros(bs).long() + self.bos_idx
        res = []
        for i in range(self.out_sl):
            h, outp = self.decoder(dec_inp, h)
            dec_inp = outp.max(1)[1]
            res.append(outp)
            if (dec_inp==self.pad_idx).all(): break
        return torch.stack(res, dim=1)
    
    def initHidden(self, bs): return one_param(self).new_zeros(self.nl,bs,self.nh)

In [ ]:
xb, yb = next(iter(data.valid_dl))

In [ ]:
xb.shape

In [ ]:
rnn = Seq2SeqRNN(emb_enc, emb_dec, 256,30)

In [ ]:
rnn

In [ ]:
len(xb[0])

In [ ]:
h = rnn.encoder(64, xb.cpu())

In [ ]:
h.size()

# Metrics

In [ ]:
def seq2seq_loss(out, targ, pad_idx=1):
    bs, targ_len = targ.size()
    _, out_len, vs  = out.size() #bs, out_len, vocab_size
    if targ_len > out_len: 
        out = F.pad(out, (0,0,0,targ_len-out_len,0,0, value=pad_idx))
    if out_len> targ_len:
        targ = F.pad(targ, (0,out_len-targ_len,0,0), value=pad_idx)
    return CrossEntropyFlat()(out, targ)

# Model

In [ ]:
learn = Learner(data, rnn, loss_func=seq2seq_loss)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4,1e-2)

In [ ]:
del fr_vecs
del en_vecs

In [ ]:
def seq2seq_acc(out, targ, pad_idx=1):
    bs, targ_len = targ.size()
    _,out_len, vs = out.size()
    if targ_len > out_len: 
        out = F.pad(out, (0,0,0,targ_len-out_len,0,0, value=pad_idx))
    if out_len> targ_len:
        targ = F.pad(targ, (0,out_len-targ_len,0,0), value=pad_idx)
    out = out.argmax(2)
    return (out==targ).float().mean()

# Bleu 

In [ ]:
class NGram():
    def __init__(self, ngram, max_n=5000):
        self.ngram,self.max_n = ngram, max_n
    def __eq__(self, other):
        if len(self.ngram)!=len(other.ngram):
            return False
    def __hash__(self):
        return int(sum([o * self.max_n**i for i,o in enumerate(self.ngram)]))

In [ ]:
def get_grams(x,n,max_n=5000)：
    return x if n==1 else [NGram(x[i:i+n], max_n=max_n) for i in range(len(x)-n+1)]


In [ ]:
def get_correct_ngrams(pred, targ, n, max_n=5000):
    pred_grams, targ_grams = get_grams(pred, n, max_n=max_n),get_grams(targ, n, max_n=max_n)
    # pred_cnt = {'word':count}
    pred_cnt, targ_cnt = Counter(pred_grams), Counter(targ_grams)
    # [min(count, targ_count)]
    return sum([min(c, targ_cnt[g]) for g,c in pred_cnt.items])

In [ ]:
class CorpusBLEU(Callback):
    def __init__(self, vocab_sz):
        self.vocab_sz, self.name = vocab_sz, 'bleu'
        
    def on_epoch_begin(self, last_output, last_target, **kwargs):
        self.pred_len, self.targ_len,self.corrects,self.counts=0.0.[0]*4,[0]*4
         
    def on_batch_end(self,last_output,last_target,**kwargs):
        last_output = last_output.argmax(dim=-1) # word
        for pred, targ in zip(last_output.cpu().numpy(), last_target.cpu().numpy()):
            self.pred_len += len(pred)
            self.targ_len += len(targ)
            # 4 grams
            for i in range(4):
                c,t = get_correct_ngrams(pred, targ, i+1, max_n=self.vocab_sz)
                self.corrects[i] += c
                self.counts[i] += t
                
    def on_epoch_end(self, last_metrics, **kwargs):
        precs = [c/t for c,t in zip(self.corrects, self.counts)]
        # penalty if prediction is incomplete
        len_penalty = exp(1-self.targ_len/self.pred_len) if self.pred < self.targ_len else 1
        # penalty * n gram^0.25
        bleu = len_penalty * ((precs[0]*precs[1]*precs[2]*precs[3])**0.25)
        return add_metrics(last_metrics, bleu)
        
        

In [ ]:
# provide answer given probability
class TeacherForcing(LearnerCallback):
    def __init__(self, learn, end_epoch):
        super().__init__(learn)
        self.end_epoch = end_epoch
    
    def on_batch_begin(self, last_input, last_target,train, **kwargs):
        if train: return {'last_input':[last_input, last_target]}
    
    def on_epoch_begin(self, epoch, **kwargs):
        self.learn.model.pr_force = 1-epoch/self.end_epoch

# Training with metrics

In [ ]:
learn = Learner(data,rnn,loss_func=seq2seq_loss, metrics=[seq2seq_acc, CorpusBLEU(len(data.y.vocab.itos))])

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, 1e-2)

In [ ]:
learn.fit_one_cycle(4, 1e-3)

# prediction

In [ ]:
def get_predictions(learn, ds_type=DatasetType.Valid):
    learn.model.eval()
    inputs, targets, outputs = [],[],[]
    with torch.no_grad():
        for xb,yb in progress_bar(learn.dl(ds_type)):
            out = learn.model(xb)
            for x,y,z in zip(xb,yb,out):
                inputs.append(learn.data.train_ds.x.reconstruct(x))
                targets.append(learn.data.train_ds.y.reconstruct(y))
                outputs.append(learn.data.train_ds.y.reconstruct(z.argmax(1)))
    return inputs, targets, outputs

# Teacher forcing

In [ ]:
class Seq2SeqRNN_tf(nn.Module):
    def __init__(self, emb_enc, emb_dec, nh, out_sl, nl=2, bos_idx=0, pad_idx=1):
        super().__init__()
        self.nl,self.nh,self.out_sl = nl,nh,out_sl
        self.bos_idx,self.pad_idx = bos_idx,pad_idx
        self.em_sz_enc = emb_enc.embedding_dim
        self.em_sz_dec = emb_dec.embedding_dim
        self.voc_sz_dec = emb_dec.num_embeddings
        
        self.emb_enc = emb_enc
        self.emb_enc_drop = nn.Dropout(0.15)
        # gru, enc_dim > nh
        # output = bs, nh
        # h = bs,nh
        self.gru_enc = nn.GRU(self.em_sz_enc, nh, num_layers=nl,
                              dropout=0.25, batch_first=True)
        # out_enc, nh > 300
        # hid = bs, 300
        self.out_enc = nn.Linear(nh, self.em_sz_dec, bias=False)
        
        self.emb_dec = emb_dec
        self.gru_dec = nn.GRU(self.em_sz_dec, self.em_sz_dec, num_layers=nl,
                              dropout=0.1, batch_first=True)
        self.out_drop = nn.Dropout(0.35)
        self.out = nn.Linear(self.em_sz_dec, self.voc_sz_dec)
        self.out.weight.data = self.emb_dec.weight.data
        self.pr_force = 0.
        
    def encoder(self, bs, inp):
        h = self.initHidden(bs)
        emb = self.emb_enc_drop(self.emb_enc(inp))
        _, h = self.gru_enc(emb, h)
        h = self.out_enc(h)
        return h
    
    def decoder(self, dec_inp, h):
        emb = self.emb_dec(dec_inp).unsqueeze(1)
        outp, h = self.gru_dec(emb, h)
        outp = self.out(self.out_drop(outp[:,0]))
        return h, outp
            
    def forward(self, inp, targ=None):
        bs, sl = inp.size()
        h = self.encoder(bs, inp)
        dec_inp = inp.new_zeros(bs).long() + self.bos_idx
        
        res = []
        for i in range(self.out_sl):
            h, outp = self.decoder(dec_inp, h)
            res.append(outp)
            dec_inp = outp.max(1)[1]
            if (dec_inp==self.pad_idx).all(): break
            # if there is a target, and random prob < pr_force
            if (targ is not None) and (random.random()<self.pr_force):
                if i>=targ.shape[1]: continue
                # input = answer
                dec_inp = targ[:,i]
        return torch.stack(res, dim=1)

    def initHidden(self, bs): return one_param(self).new_zeros(self.nl, bs, self.nh)

In [ ]:
emb_enc = torch.load(model_path/'fr_emb.pth')
emb_dec = torch.load(model_path/'en_emb.pth')

In [ ]:
rnn_tf = Seq2SeqRNN_tf(emb_enc, emb_dec, 256, 30)

learn = Learner(data, rnn_tf, loss_func=seq2seq_loss, metrics=[seq2seq_acc, CorpusBLEU(len(data.y.vocab.itos))],
               callback_fns=partial(TeacherForcing, end_epoch=3))

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(6, 3e-3)